In [1]:
%matplotlib qt

import time
import copy
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import cPickle as pickle

import matplotlib.pyplot as plt

from spimagine import volfig, volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d import NetSurf3d
from data4d import Data4d

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


# Load time-lapse data...

In [2]:
filenames = ['/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t1.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t2.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t3.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t4.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t5.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t6.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t7.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t8.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t9.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t10.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t11.TIF',
             '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t12.TIF']

In [3]:
data = Data4d( filenames, pixelsize=(1.,1.,5.), silent=False )
data.set_seg_params(K=30,max_delta_k=4)
objects = []
ecells = []

Dimensions of frame 0 :  (31, 512, 512)
Dimensions of frame 1 :  (31, 512, 512)
Dimensions of frame 2 :  (31, 512, 512)
Dimensions of frame 3 :  (31, 512, 512)
Dimensions of frame 4 :  (31, 512, 512)
Dimensions of frame 5 :  (31, 512, 512)
Dimensions of frame 6 :  (31, 512, 512)
Dimensions of frame 7 :  (31, 512, 512)
Dimensions of frame 8 :  (31, 512, 512)
Dimensions of frame 9 :  (31, 512, 512)
Dimensions of frame 10 :  (31, 512, 512)
Dimensions of frame 11 :  (31, 512, 512)


In [4]:
max_rs = (data.images[0].shape[2] / 8,
          data.images[0].shape[1] / 8,
          data.images[0].shape[0] / 2)

min_rs = (5,5,1)

## adding E cells

In [5]:
Ea = data.init_object("Ea")
objects.append(Ea)
ecells.append(Ea)

Ep = data.init_object("Ep")
objects.append(Ep)
ecells.append(Ep)

In [6]:
# data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(110,300,12), frame_to=11, seed_to=(105,310,16), segment_it=True )
data.add_object_at( Ea, min_rs, max_rs, frame=0, seed=(110,300,12), segment_it=True )
data.track(Ea, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ea" in frame 0 with seed coordinates [ 110.  300.   12.]
      Optimum energy:  95253.0
      Nodes in/out:  7349 8011
Added appearance for "Ea" in frame 1 with seed coordinates [ 110.  300.   12.]
      Optimum energy:  93961.0
      Nodes in/out:  7543 7817
    Updated center to [110 301  13]
Added appearance for "Ea" in frame 2 with seed coordinates [ 110.  301.   13.]
      Optimum energy:  82339.0
      Nodes in/out:  8052 7308
    Updated center to [111 301  13]
Added appearance for "Ea" in frame 3 with seed coordinates [ 111.  301.   13.]
      Optimum energy:  79644.0
      Nodes in/out:  8384 6976
    Updated center to [112 302  13]
Added appearance for "Ea" in frame 4 with seed coordinates [ 112.  302.   13.]
      Optimum energy:  77683.0
      Nodes in/out:  8404 6956
    Updated center to [114 302  13]
Added appearance for "Ea" in frame 5 with seed coordinates [ 114.  302.   13.]
      Optimum energy:  77008.0
      Nodes in/out:  8282 7078
    Update

In [7]:
data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), frame_to=11, seed_to=(140,360,16), segment_it=True )
# data.add_object_at( Ep, min_rs, max_rs, frame=0, seed=(110,350,12), segment_it=True )
# data.track(Ep, seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "Ep" in frame 0 with seed coordinates [ 110.  350.   12.]
      Optimum energy:  87730.0
      Nodes in/out:  7060 8300
Added appearance for "Ep" in frame 1 with seed coordinates [ 112.  351.   12.]
      Optimum energy:  85070.0
      Nodes in/out:  7123 8237
Added appearance for "Ep" in frame 2 with seed coordinates [ 115.  352.   13.]
      Optimum energy:  83739.0
      Nodes in/out:  7540 7820
Added appearance for "Ep" in frame 3 with seed coordinates [ 118.  352.   13.]
      Optimum energy:  84401.0
      Nodes in/out:  7321 8039
Added appearance for "Ep" in frame 4 with seed coordinates [ 120.  353.   13.]
      Optimum energy:  81773.0
      Nodes in/out:  7064 8296
Added appearance for "Ep" in frame 5 with seed coordinates [ 122.  354.   14.]
      Optimum energy:  77667.0
      Nodes in/out:  7152 8208
Added appearance for "Ep" in frame 6 with seed coordinates [ 125.  355.   14.]
      Optimum energy:  77966.0
      Nodes in/out:  6948 8412
Added appeara

## adding direct neighbors

In [8]:
neighbors = []
for i in range(12):
    n = data.init_object("n"+str(i))
    objects.append(n)
    neighbors.append(n)

In [9]:
nseeds = [( 90,390,12),
          (150,410,12),
          (200,390,12),
          (190,315,12),
          (145,265,12),
          (110,230,12),
          ( 40,255,12),
          ( 45,300,18),
          ( 67,355,18),
          (110,275,24),
          (130,315,24),
          (125,360,24)]
for i in range(len(nseeds)):
    data.add_object_at( neighbors[i], min_rs, max_rs, frame=0, seed=nseeds[i], segment_it=True )
    data.track(neighbors[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "n0" in frame 0 with seed coordinates [  90.  390.   12.]
      Optimum energy:  75930.0
      Nodes in/out:  4803 10557
Added appearance for "n0" in frame 1 with seed coordinates [  90.  390.   12.]
      Optimum energy:  74323.0
      Nodes in/out:  4958 10402
    Updated center to [ 91 387  12]
Added appearance for "n0" in frame 2 with seed coordinates [  91.  387.   12.]
      Optimum energy:  73773.0
      Nodes in/out:  5196 10164
    Updated center to [ 90 384  12]
Added appearance for "n0" in frame 3 with seed coordinates [  90.  384.   12.]
      Optimum energy:  74514.0
      Nodes in/out:  5173 10187
    Updated center to [ 89 381  11]
Added appearance for "n0" in frame 4 with seed coordinates [  89.  381.   11.]
      Optimum energy:  72659.0
      Nodes in/out:  5013 10347
    Updated center to [ 89 378  11]
Added appearance for "n0" in frame 5 with seed coordinates [  89.  378.   11.]
      Optimum energy:  72933.0
      Nodes in/out:  5167 10193
    

## adding others

In [10]:
others = []
for i in range(7):
    o = data.init_object("o"+str(i))
    objects.append(o)
    others.append(o)

In [11]:
oseeds = [( 70,205,12),
          (145,170,12),
          ( 50,160,12),
          (105,140,12),
          (190,220,12),
          (200,265,12),
          (220,335,20)]
for i in range(len(oseeds)):
    data.add_object_at( others[i], min_rs, max_rs, frame=0, seed=oseeds[i], segment_it=True )
    data.track(others[i], seed_frame=0, target_frames=range(1,len(data.images)), recenter_iterations=1)

Added appearance for "o0" in frame 0 with seed coordinates [  70.  205.   12.]
      Optimum energy:  79859.0
      Nodes in/out:  7453 7907
Added appearance for "o0" in frame 1 with seed coordinates [  70.  205.   12.]
      Optimum energy:  79123.0
      Nodes in/out:  7182 8178
    Updated center to [ 67 204  11]
Added appearance for "o0" in frame 2 with seed coordinates [  67.  204.   11.]
      Optimum energy:  78390.0
      Nodes in/out:  7041 8319
    Updated center to [ 66 204  11]
Added appearance for "o0" in frame 3 with seed coordinates [  66.  204.   11.]
      Optimum energy:  77943.0
      Nodes in/out:  6966 8394
    Updated center to [ 66 203  11]
Added appearance for "o0" in frame 4 with seed coordinates [  66.  203.   11.]
      Optimum energy:  76368.0
      Nodes in/out:  6878 8482
    Updated center to [ 65 203  11]
Added appearance for "o0" in frame 5 with seed coordinates [  65.  203.   11.]
      Optimum energy:  74884.0
      Nodes in/out:  6739 8621
    Update

# Save / Load

In [12]:
data.save('/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry.pkl')

In [ ]:
data = Data4d( [], silent=False )
data.load('/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry.pkl')
objects = range(len(data.object_names))
ecells = [0,1]
neighbors = range(2,2+12)
others = range(2+12,2+12+7)

# Visualisation stuff 

In [13]:
w = data.show_frame(0,stackUnits=[1.,1.,5.],raise_window=True)
data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)

In [14]:
for f in range(len(data.images)):
    print 'Exporting frame ',f+1,'of',len(data.images)
    w = data.show_frame(f,stackUnits=[1.,1.,5.],raise_window=False)
    data.show_objects(ecells, show_surfaces=True, show_centers=True, colors=data.colors_gold)
    data.show_objects(neighbors, show_surfaces=True, show_centers=True, colors=data.colors_green)
    data.show_objects(others, show_surfaces=True, show_centers=True, colors=data.colors_blue)
    data.save_current_visualization( 'frame'+str(f)+'.jpg' )

Exporting frame  1 of 12
Exporting frame  2 of 12
Exporting frame  3 of 12
Exporting frame  4 of 12
Exporting frame  5 of 12
Exporting frame  6 of 12
Exporting frame  7 of 12
Exporting frame  8 of 12
Exporting frame  9 of 12
Exporting frame  10 of 12
Exporting frame  11 of 12
Exporting frame  12 of 12
